In [11]:
 # Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd

In [12]:
csv_file_path = ("../Car price data/accord_data.csv")

In [13]:
# Using Splinter Scraping data
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
# URL of page to be scraped
url = 'https://losangeles.craigslist.org/search/cto?min_price=2000&auto_make_model=accord&min_auto_year=2008&max_auto_year=2018&min_auto_miles=0&max_auto_miles=1000000&condition=10&condition=20&condition=30&condition=40'

In [14]:
# Scrape the data based on post info
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html,"html.parser")
next_page = soup.find("a",class_ = "button next")["href"]

titles = []
prices = []
links = []
car_data = []

while next_page:
    print(" - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - ")
    html = browser.html
    soup = BeautifulSoup(html,"html.parser")
    results = soup.find_all('li', class_="result-row")
    
    
    
    
    for result in results:
    # Error handling
        try:
            # Identify and return title of listing
            title = result.find('a', class_="result-title").text
            # Identify and return price of listing
            price = result.a.span.text
            # Identify and return link to listing
            link = result.a['href']

            # Print results only if title, price, and link are available
            if (title and price and link):
                titles.append(title)
                prices.append(price)
                links.append(link)
                
                post_link = result.a["href"]
                response2 = requests.get(post_link)
                soup2 = BeautifulSoup(response2.text,"html.parser")
                attributes = soup2.find_all("p",class_="attrgroup")
                attribute_list = attributes[1].find_all("span")
                attr_dic = {}
                
                try:
                
                    tag = attributes[0].find("span").find("b").text.split(" ")
                
                    yearlist = [str(num) for num in list(range(1900, 2020))]
                    for word in tag:
                        if word in yearlist:
                            year = word
                            for attr in attribute_list:
                                try:
                                    key,value = attr.text.split(":")
                                    attr_dic[key] = value
                                    attr_dic["Year"] = year
                                except ValueError:
                                    pass
                            if word not in tag:
                                attr_dic["Year"] = "NA"
                    
                    
                except:
                    attr_dic["Year"] = "NA"
                
    
                car_data.append(attr_dic)
                
                
        except AttributeError as e:
            print(e)
    try:
        browser.click_link_by_partial_text('next')
        next_page = soup.find("a",class_ = "button next")["href"]
    except:
        print("- - - - - - - - - This is the last page we can find!!!- - - - - -")
        next_page = False

 - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - 
 - - - - - - - - - - - Scraping New Page Starts!! - - - - - - - - - 
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
- - - - - - - - - This is the last page we can find!!!- - - - - -


In [16]:
# Store info as dataframe
info_df = pd.DataFrame({
    "Info" : titles,
    "Price" : prices,
    "links" : links
})

In [17]:
# Show full dataframe
pd.set_option('display.max_colwidth', -1)
# Merge two dataframe
car_df = pd.DataFrame(car_data)
car_df["Title"] = info_df["Info"]
car_df["Price"] = info_df["Price"]
car_df["Link"] = info_df["links"]
car_df["Make"] = "Honda"
car_df["Model"] = "Accord"
car_df = car_df[['Title','Make', 'Model','Year', 'condition', 'cylinders', 'drive', 'fuel', 'odometer',
       'paint color', 'size', 'title status', 'transmission', 'type', 
       'Price', 'VIN', 'Link']]
car_df.head(50)

,Title,Make,Model,Year,condition,cylinders,drive,fuel,odometer,paint color,size,title status,transmission,type,Price,VIN,Link
0,Honda Accord,Honda,Accord,2010,good,4 cylinders,fwd,gas,117000,grey,mid-size,salvage,automatic,sedan,$5800,1hgcp2f34aa019623,https://losangeles.craigslist.org/lgb/cto/d/honda-accord/6710644067.html
1,2012 Honda accord coupe,Honda,Accord,2012,excellent,4 cylinders,NaN,gas,111,NaN,NaN,clean,automatic,NaN,$7995,NaN,https://losangeles.craigslist.org/wst/cto/d/2012-honda-accord-coupe/6699535701.html
2,2017 Honda Acoord Sport,Honda,Accord,2017,good,4 cylinders,fwd,gas,22,black,mid-size,salvage,automatic,sedan,$14500,NaN,https://losangeles.craigslist.org/lgb/cto/d/2017-honda-acoord-sport/6703846822.html
3,2017 Honda Accord sport special edition clean tittle,Honda,Accord,2017,like new,4 cylinders,fwd,gas,2017,black,NaN,clean,automatic,sedan,$18900,NaN,https://losangeles.craigslist.org/wst/cto/d/2017-honda-accord-sport/6710581560.html
4,2011 Honda Accord Exl,Honda,Accord,2011,good,4 cylinders,fwd,gas,82500,black,mid-size,clean,automatic,sedan,$8300,1HGCP2F88BA080938,https://losangeles.craigslist.org/sfv/cto/d/2011-honda-accord-exl/6710548793.html
5,"* 2009 Honda Accord LX-p 4Door Sedan 100K * NO ACCIDENT ,PERFE RUN !!!",Honda,Accord,2009,excellent,4 cylinders,NaN,gas,100000,NaN,NaN,clean,automatic,NaN,$6450,1HGCP26499A197306,https://losangeles.craigslist.org/lac/cto/d/2009-honda-accord-lx-4door/6694133929.html
6,2017 Honda Accord EX-L V6 Clean Title,Honda,Accord,2017,like new,6 cylinders,fwd,gas,21000,grey,mid-size,clean,automatic,sedan,$23900,NaN,https://losangeles.craigslist.org/sfv/cto/d/2017-honda-accord-ex-v6-clean/6704045437.html
7,2012 Honda Accord EX-L - $10900,Honda,Accord,2012,excellent,4 cylinders,rwd,gas,72000,NaN,NaN,clean,automatic,sedan,$10900,NaN,https://losangeles.craigslist.org/wst/cto/d/2012-honda-accord-ex-10900/6710429893.html
8,Honda Accord 2015 palmdale,Honda,Accord,2015,like new,4 cylinders,NaN,gas,61000,NaN,NaN,salvage,automatic,NaN,$9500,NaN,https://losangeles.craigslist.org/ant/cto/d/honda-accord-2015-palmdale/6710420915.html
9,"Honda Accord EX Sedan (56,900 Miles) Clean Title, Clean Carfax,1 Owner",Honda,Accord,2014,like new,4 cylinders,NaN,gas,56900,black,NaN,clean,automatic,sedan,$13400,1HGCR2F72EA172102,https://losangeles.craigslist.org/lac/cto/d/honda-accord-ex-sedanmiles/6692302788.html


In [18]:
# Out put as CSV file
car_df.to_csv(csv_file_path)